In [1]:
import pathlib
import shutil
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm, trange
from image_processing import load_image, convert_nifti_batch_to_dicom
from image_processing import ImageProcess
from image_processing import read_dicom_series
from image_processing.util import save_file_path_to_pandas_table

<Figure size 432x288 with 0 Axes>

### Step 1
Statistics of file paths in the destination directory
If you have the pandas table of paths, then skip this step 

In [2]:
origin_data_home = '/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data'
new_data_home = '/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data_in_dicoms'
save_file_path_to_pandas_table(origin_data_home, pathlib.Path(new_data_home).parent/'nifti_path_info.csv')
info_df = pd.read_csv(pathlib.Path(new_data_home).parent/'nifti_path_info.csv')
info_df['out_file_path'] = info_df['file_path'].apply(lambda x: x.replace('composite_data', 'composite_data_in_dicoms').replace('.nii.gz',''))
info_df.head()
info_df['out_file_path'][0]

=  File paths saved to /media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/nifti_path_info.csv  =


'/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data_in_dicoms/1.5/宋照英/composite_1_3'

### Setp 2
Covert nifti files into dicoms

In [ ]:
def convert_nifti_batch_to_dicom(info_table, nifti_col, dicom_col, overwrite=True):
    for ind, row in tqdm(info_table.iterrows()):
        nii_file_path = pathlib.Path(row[nifti_col])
        dicom_dir_path = pathlib.Path(row[dicom_col])
        if not overwrite:
            if dicom_dir_path.exists():
                continue
        else:
            if dicom_dir_path.exists():
                shutil.rmtree(dicom_dir_path, ignore_errors=True)
        imgprs = ImageProcess(nii_file_path)
        imgprs.saveimg(savepath=dicom_dir_path, format='dcm', meta_data=False)

In [11]:
convert_nifti_batch_to_dicom(info_df, 'file_path', 'out_file_path', overwirte=False)

210it [08:43,  2.49s/it]


### Step 3
Check the information of the new dicoms series

In [5]:
example_nifti_path = info_df['file_path'][0]
example_dicoms_path = info_df['out_file_path'][0]
dicoms_path = pathlib.Path(example_dicoms_path)
reader = sitk.ImageSeriesReader()
dicomName = reader.GetGDCMSeriesFileNames(str(dicoms_path))
reader.SetFileNames(dicomName)
sitk_image = reader.Execute()

In [6]:
sitk_image.GetSize()

(512, 512, 306)

In [7]:
nii_sitk_image = load_image(example_nifti_path)
nii_sitk_image.GetSize()

(512, 512, 306)

In [8]:
sitk_image.GetSpacing()

(0.711000025272369, 0.711000025272369, 1.0)

In [9]:
nii_sitk_image.GetSpacing()

(0.7110000252723694, 0.7110000252723694, 1.0)